# Libraries
1. [filterpy](https://filterpy.readthedocs.io/en/latest/kalman/UnscentedKalmanFilter.html)

In [1]:
import numpy as np
import pandas as pd
import random
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px
import seaborn as sns

import geopy
import pymap3d as pm
from filterpy.kalman import UnscentedKalmanFilter, MerweScaledSigmaPoints

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchsummary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings(action='ignore')

# Hyper Parameters

In [2]:
SEED = 1990
random.seed(SEED)
np.random.seed(SEED)

In [3]:
q = np.array([1,1,1,1,1])
r = np.array([1,1,1,1])

# Useful functions

In [4]:
def calc_haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    
    c = 2 * np.arcsin(a ** 0.5)
    dist = 6_367_000 * c
    return dist

In [5]:
def check_score(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.t_latDeg, r.t_lngDeg
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    
    return output_df, meter_score , score

In [6]:
ell_wgs84 = pm.Ellipsoid()
def calc_geo2enu(df:pd.DataFrame)->pd.DataFrame:
    output = df.copy()
    llh = np.array(df[['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']])
    denu = pm.geodetic2enu(llh[:,0], llh[:,1], llh[:,2], llh[0,0], llh[0,1], llh[0,2], ell=ell_wgs84)
    output['x'] = denu[0]
    output['y'] = denu[1]
    output['z'] = denu[2]
    
    return output

def calc_enu2geo(df:pd.DataFrame)->pd.DataFrame:
    output = df.copy()
    enu = np.array(df[['x', 'y', 'z']])
    llh = np.array(df[['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']])
    geo = pm.enu2geodetic(enu[:,0], enu[:,1], enu[:,2], llh[0,0], llh[0,1], llh[0,2], ell=ell_wgs84, deg = True)
    output['latDeg'] = geo[0]
    output['lngDeg'] = geo[1]
    output['heightAboveWgs84EllipsoidM'] = geo[2]
    
    return output

# Data

In [7]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")
df_train = pd.read_pickle(str(data_dir / "gsdc_extract_train.pkl.gzip"))
df_test = pd.read_pickle(str(data_dir / "gsdc_extract_test.pkl.gzip"))

In [8]:
phones = df_train['phone'].unique()
phone = phones[random.randint(0, len(phones))]
df_sample = df_train[df_train['phone'] == phone].copy().reset_index().drop(columns = ['index'])
print(df_sample.shape)
df_sample.head()

(1746, 148)


,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,timeSinceFirstFixSeconds,hDop,vDop,...,GPS_L1,GPS_L5,GAL_E1,GAL_E5A,GLO_G1,BDS_B1I,BDS_B1C,BDS_B2A,QZS_J1,QZS_J5
0,2020-05-14-US-MTV-1,Pixel4XLModded,1273529466449,37.423574,-122.094137,-33.20,2020-05-14-US-MTV-1_Pixel4XLModded,554.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2020-05-14-US-MTV-1,Pixel4XLModded,1273529467449,37.423573,-122.094153,-33.92,2020-05-14-US-MTV-1_Pixel4XLModded,555.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2020-05-14-US-MTV-1,Pixel4XLModded,1273529468449,37.423578,-122.094148,-33.33,2020-05-14-US-MTV-1_Pixel4XLModded,556.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2020-05-14-US-MTV-1,Pixel4XLModded,1273529469449,37.423573,-122.094150,-32.85,2020-05-14-US-MTV-1_Pixel4XLModded,557.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2020-05-14-US-MTV-1,Pixel4XLModded,1273529470449,37.423573,-122.094147,-31.26,2020-05-14-US-MTV-1_Pixel4XLModded,558.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
for col in df_train.columns:
    print(col)

collectionName
phoneName
millisSinceGpsEpoch
latDeg
lngDeg
heightAboveWgs84EllipsoidM
phone
timeSinceFirstFixSeconds
hDop
vDop
speedMps
courseDegree
t_latDeg
t_lngDeg
t_heightAboveWgs84EllipsoidM
constellationType
svid
signalType
receivedSvTimeInGpsNanos
xSatPosM
ySatPosM
zSatPosM
xSatVelMps
ySatVelMps
zSatVelMps
satClkBiasM
satClkDriftMps
rawPrM
rawPrUncM
isrbM
ionoDelayM
tropoDelayM
utcTimeMillis
elapsedRealtimeNanos
yawDeg
rollDeg
pitchDeg
utcTimeMillis_Status
SignalCount
SignalIndex
ConstellationType
Svid
CarrierFrequencyHz
Cn0DbHz
AzimuthDegrees
ElevationDegrees
UsedInFix
HasAlmanacData
HasEphemerisData
BasebandCn0DbHz
utcTimeMillis_UncalMag
elapsedRealtimeNanos_UncalMag
UncalMagXMicroT
UncalMagYMicroT
UncalMagZMicroT
BiasXMicroT
BiasYMicroT
BiasZMicroT
utcTimeMillis_UncalAccel
elapsedRealtimeNanos_UncalAccel
UncalAccelXMps2
UncalAccelYMps2
UncalAccelZMps2
BiasXMps2
BiasYMps2
BiasZMps2
utcTimeMillis_UncalGyro
elapsedRealtimeNanos_UncalGyro
UncalGyroXRadPerSec
UncalGyroYRadPerSec
U

## Model Define
$$
\begin{matrix}
x_t =& x_{t-1} + \frac{v_{t-1}}{w_{t-1}}\left({\sin}\left({\omega}_{t-1}dt + {\theta}_{t-1}\right) - {\sin}\left({\theta}\right)\right)\\
y_t =& y_{t-1} + \frac{v_{t-1}}{w_{t-1}}\left({\cos}\left({\theta}_{t-1}\right) - {\cos}\left({\omega}_{t-1}dt + {\theta}_{t-1}\right)\right)\\
v_t =& v_{t-1}\\
{\theta}_t =& {\theta}_{t-1} + {\omega}_{t-1}dt\\
{\omega}_t =& {\omega}_{t-1}
\end{matrix}
$$

In [10]:
def fx(x, dt):
    xout = np.zeros_like(x)
    if abs(x[4]) > 1e-3:
        xout[0] = x[0] + x[2]/x[4] * (np.sin(x[4] * dt + x[3]) - np.sin(x[3]))
        xout[1] = x[1] + x[2]/x[4] * (np.cos(x[3]) - np.cos(x[4] * dt + x[3]))
        xout[2] = x[2]
        xout[3] = x[3] + x[4] * dt
        xout[4] = x[4]
    else:
        xout[0] = x[0] + x[2] * dt * (np.cos(x[3]))
        xout[1] = x[1] + x[2] * dt * (np.sin(x[3]))
        xout[2] = x[2]
        xout[3] = x[3] + x[4] * dt
        xout[4] = x[4]
        
    return xout

In [11]:
def batch_filter(df, q_, r_):
    df1 = calc_geo2enu(df)
    df1['yawRad'] = np.deg2rad(df1['yawDeg'])
    
    
    features = ['x', 'y']
    index = [0, 1]
    rindex = [0, 1]
    if df1['yawDeg'].isna().mean() == 1 or df1['yawDeg'].mean() == 0:
        pass
    else:
        features += ['yawRad']
        index += [3]
        rindex+= [2]
    if df1['UncalGyroZRadPerSec'].isna().mean() == 1 or df1['UncalGyroZRadPerSec'].mean() == 0:
        pass
    else:
        features += ['UncalGyroZRadPerSec']
        index += [4]
        rindex+= [3]
    
    q = q_
    r = r_[rindex]
    meas = df1[features]
    meas = meas.fillna(0)
    h = lambda x:x[index]
    
    points = MerweScaledSigmaPoints(5, alpha = .1, beta = 2., kappa = -1)
    kf = UnscentedKalmanFilter(dim_x = 5, dim_z = len(features), dt = 1, fx = fx, hx = h, points = points)
    kf.Q = np.diag(q)
    kf.R = np.diag(r)
    
    mu, cov = kf.batch_filter(meas.values)
    (xs, Ps, Ks) = kf.rts_smoother(mu, cov)
    df2 = df1.copy()
    df2['x'] = xs[:,0]
    df2['y'] = xs[:,1]
    df2['yawDeg'] = np.rad2deg(xs[:,2])
    df2['UncalGyroZRadPerSec'] = xs[:,3]
    df3 = calc_enu2geo(df2)
    return df3

In [12]:
def evaluate(df, q_, r_, get_score = True):
    output = df.copy()
    mean_before, score_before, mean_after, score_after =0, 0, 0, 0
    output.drop(columns = ['latDeg', 'lngDeg'], inplace = True)
    df_list = []
    
    for phone in tqdm(df['phone'].unique()):
        df1 = df[df['phone'] == phone].copy()
        df2 = batch_filter(df1, q_, r_)
        df_list.append(df2[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']])
    df3 = pd.concat(df_list)
    output = output.merge(df3, on = ['phone', 'millisSinceGpsEpoch'])

    if get_score:
        _, mean_before, score_before = check_score(df)
        _, mean_after, score_after = check_score(output)
    
    return output, mean_before, score_before, mean_after, score_after

In [17]:
q_gain_list = np.arange(0.1, 2, 0.1)
r_gain_list = np.arange(0.1, 2, 0.1)

q_ = np.array([0.01, 0.01, 0.05, 0.5*np.pi/180, 2.5*np.pi/180])**2
r_ = np.array([0.01, 0.01, 0.5*np.pi/180, 2.5*np.pi/180])**2
phones = df_train['phone'].unique()
sample_phone = np.random.choice(phones, 5, replace = False)
sample_index = df_train['phone'].apply(lambda x: x in sample_phone)


result_list = []
iter_count = 0
for r_gain in r_gain_list:
    for q_gain in q_gain_list:
        iter_count += 1
        q = q_gain * q_
        r = r_gain * r_
        _, mean_before, score_before, mean_after, score_after = evaluate(df_train[sample_index], q, r)
        result = [q, r, mean_after, mean_before, score_after, score_before]
        print(f"{q_gain}, {r_gain} : mean chagne - {mean_after - mean_before:.6f}, score change - {score_after - score_before:.6f}")
        print(f"after[{mean_after}, {score_after}], before[{mean_before}, {score_before}]")
        print(q_gain * q_)
        print(r_gain * r_)
        result_list.append(result)

  0%|          | 0/5 [00:00<?, ?it/s]

0.1, 0.1 : mean chagne - 0.047109, score change - 0.169391
after[3.713826479249931, 7.203849090006893], before[3.666717828362788, 7.034457791343084]
[1.00000000e-05 1.00000000e-05 2.50000000e-04 7.61543549e-06
 1.90385887e-04]
[1.00000000e-05 1.00000000e-05 7.61543549e-06 1.90385887e-04]


  0%|          | 0/5 [00:00<?, ?it/s]

0.2, 0.1 : mean chagne - 208.390893, score change - 840.194236
after[212.05761130559156, 847.2286940331269], before[3.666717828362788, 7.034457791343084]
[2.00000000e-05 2.00000000e-05 5.00000000e-04 1.52308710e-05
 3.80771775e-04]
[1.00000000e-05 1.00000000e-05 7.61543549e-06 1.90385887e-04]


  0%|          | 0/5 [00:00<?, ?it/s]

...

1.7000000000000002, 1.9000000000000001 : mean chagne - 6.141247, score change - 23.398965
after[9.807964428854811, 30.433422800784076], before[3.666717828362788, 7.034457791343084]
[0.00017    0.00017    0.00425    0.00012946 0.00323656]
[0.00019    0.00019    0.00014469 0.00361733]


  0%|          | 0/5 [00:00<?, ?it/s]

1.8000000000000003, 1.9000000000000001 : mean chagne - 0.223450, score change - 0.489419
after[3.890167834955804, 7.523877015088426], before[3.666717828362788, 7.034457791343084]
[0.00018    0.00018    0.0045     0.00013708 0.00342695]
[0.00019    0.00019    0.00014469 0.00361733]


  0%|          | 0/5 [00:00<?, ?it/s]

1.9000000000000001, 1.9000000000000001 : mean chagne - 0.088139, score change - 0.212766
after[3.7548566268145462, 7.2472239028377174], before[3.666717828362788, 7.034457791343084]
[0.00019    0.00019    0.00475    0.00014469 0.00361733]
[0.00019    0.00019    0.00014469 0.00361733]


In [13]:
q_gain_list = np.arange(0.1, 2, 0.1)
r_gain_list = np.arange(0.1, 2, 0.1)

q_ = np.array([0.01, 0.01, 0.1, 1*np.pi/180, 10*np.pi/180])**2
r_ = np.array([0.1, 0.1, 5 * np.pi/180, 20 * np.pi/180])**2
phones = df_train['phone'].unique()


result_list = []
iter_count = 0
for q_gain in q_gain_list:
    for r_gain in r_gain_list:
        iter_count += 1
        q = q_gain * q_
        r = r_gain * r_
#         sample_phone = np.random.choice(phones, 13, replace = False)
#         sample_index = df_train['phone'].apply(lambda x: x in sample_phone)
#         _, mean_before, score_before, mean_after, score_after = evaluate(df_train[sample_index], q, r)
        _, mean_before, score_before, mean_after, score_after = evaluate(df_train, q, r)
        result = [q, r, mean_after, mean_before, score_after, score_before]
        print(f"{q_gain}, {r_gain} : mean chagne - {mean_after - mean_before:.6f}, score change - {score_after - score_before:.6f}")
        print(f"after[{mean_after}, {score_after}], before[{mean_before}, {score_before}]")
        result_list.append(result)

  0%|          | 0/73 [00:00<?, ?it/s]

0.1, 0.1 : mean chagne - 3.175018, score change - 7.199920
after[7.021866761818569, 12.487890582667237], before[3.84684837499064, 5.2879706490841585]


  0%|          | 0/73 [00:00<?, ?it/s]

0.1, 0.2 : mean chagne - 3.786547, score change - 9.053881
after[7.6333954147533705, 14.34185213488532], before[3.84684837499064, 5.2879706490841585]


  0%|          | 0/73 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
df = pd.DataFrame(result_list, columns = ['q', 'r', 'mean_after', 'mean_before', 'score_after', 'score_before'])


In [21]:
df['qgain'] = df['q'].apply(lambda x: x.mean())
df['rgain'] = df['r'].apply(lambda x: x.mean())
df['change_score'] = df['score_after'] - df['score_before']

In [ ]:
pt = df.pivot_table('change_score', 'qgain', 'rgain')
sns.heatmap(pt)

In [24]:
q = df.loc[df['change_score'].min() == df['change_score'], 'q'].values[0]
r = df.loc[df['change_score'].min() == df['change_score'], 'r'].values[0]
print(q, r)

[1.10000000e-04 1.10000000e-04 2.75000000e-03 8.37697904e-05
 2.09424476e-03] [8.0000000e-05 8.0000000e-05 6.0923484e-05 1.5230871e-03]


In [ ]:
_, mean_before, score_before, mean_after, score_after =  evaluate(df_train, q, r)
print(mean_before, score_before, mean_after, score_after)
print(f"mean change : {mean_after - mean_before:.4f}")
print(f"score change: {score_after - score_before:.4f}")

  0%|          | 0/73 [00:00<?, ?it/s]

In [ ]:
submission = pd.read_csv("../input/google-smartphone-decimeter-challenge/sample_submission.csv")
submission = submission[['phone', 'millisSinceGpsEpoch']]

In [ ]:
result, _, _, _, _ =  evaluate(df_test, q, r, get_score=False)
result = result[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']]
submission = submission.merge(result, on = ['phone', 'millisSinceGpsEpoch'])

In [ ]:
submission.to_csv(f"./models/{'ComplexKalmanFilter1'}/result-{4}result.csv", index = False)